In [ ]:
import { OpenAIInstrumentation } from "npm:@arizeai/openinference-instrumentation-openai";
import { ConsoleSpanExporter } from "npm:@opentelemetry/sdk-trace-base";
import {
  NodeTracerProvider,
  SimpleSpanProcessor,
} from "npm:@opentelemetry/sdk-trace-node";
import { Resource } from "npm:@opentelemetry/resources";
import { OTLPTraceExporter } from "npm:@opentelemetry/exporter-trace-otlp-proto";
import { SemanticResourceAttributes } from "npm:@opentelemetry/semantic-conventions";
import { diag, DiagConsoleLogger, DiagLogLevel } from "npm:@opentelemetry/api";

// For troubleshooting, set the log level to DiagLogLevel.DEBUG
diag.setLogger(new DiagConsoleLogger(), DiagLogLevel.DEBUG);

const provider = new NodeTracerProvider({
  resource: new Resource({
    [SemanticResourceAttributes.SERVICE_NAME]: "openai-service",
  }),
});

provider.addSpanProcessor(new SimpleSpanProcessor(new ConsoleSpanExporter()));
provider.addSpanProcessor(
  new SimpleSpanProcessor(
    new OTLPTraceExporter({
      url: "http://localhost:6006/v1/traces",
    }),
  ),
);

const oaiInstrumentor = new OpenAIInstrumentation();

provider.register();

console.log("👀 OpenInference initialized");

In [ ]:
import OpenAI from 'npm:openai';
oaiInstrumentor.manuallyInstrument(OpenAI);

In [ ]:
// import {getPass} from 'npm:getpass';

// const apiKey = await new Promise((resolve, reject) => {
//   const existingApiKey = Deno.env.get('OPENAI_API_KEY');
//    if (!existingApiKey) { 
//     getPass({prompt: "Enter your OpenAI API key 🔑"}, (err: Error | undefined, pass: string) => {
//       if (err) {
//         reject(err);
//       }
//       resolve(pass);
//     });
//   } else {
//     resolve(existingApiKey);
//   }
// });

// import getpass from "https://deno.land/x/getpass/mod.ts";

// const apiKey = getpass();

let apiKey: string | undefined | null = Deno.env.get('OPENAI_API_KEY');

if (!apiKey) {
  apiKey = prompt("Enter your OpenAI API key 🔑");
}

if (!apiKey) {
  throw new Error("No OpenAI API key found");
}


In [ ]:
const client = new OpenAI({
  apiKey,
});

async function main() {
 try {
  const chatCompletion = await client.chat.completions.create({
    messages: [{ role: 'user', content: 'Say this is a test' }],
    model: 'gpt-3.5-turbo',
  });
  console.dir(chatCompletion.choices[0].message);
 } catch (e) {
   console.error(e);
 }
}

await main();